## 1 ère étape : Créer un fichier csv  avec les coordonnées des landmarks 
--



Les lignes du représentent les coordonnées des points faciaux (landmarks) détectés par le modèle Mediapipe FaceMesh.
- landmark_index: Il s'agit de l'indice du point facial (landmark) dans la liste des landmarks détectés. 
- X: C'est la coordonnée X du landmark. 
- Y: C'est la coordonnée Y du landmark.



## a) Récupérer les coordonnées d'une vidéo dans un csv (ne pas exécuter cette cellule)

In [ ]:
import cv2
import mediapipe as mp
import csv
import os

# Ajoutez cette ligne pour spécifier la version de numpy
import numpy as np; np_version = np.__version__.split('.')

if int(np_version[0]) < 1 or (int(np_version[0]) == 1 and int(np_version[1]) < 20):
    raise ImportError("Numpy version 1.20.0 or above is required for this version of mediapipe. "
                      "Please upgrade numpy by running: pip install --upgrade numpy")



# Définition de la fonction eye_aspect_ratio
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Définition de la fonction mouth_aspect_ratio
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[1] - mouth[5])
    B = np.linalg.norm(mouth[2] - mouth[4])
    C = np.linalg.norm(mouth[0] - mouth[3])
    mar = (A + B) / (2.0 * C)
    return mar

# Charger le modèle Mediapipe pour la détection des yeux et des lèvres
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Initialiser le détecteur de visage et le dessinateur pour les annotations
face_mesh = mp_face_mesh.FaceMesh()
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Charger la vidéo
video_path = 'kss#8-9#F#rldd#28-10.mp4'
video_name = os.path.splitext(os.path.basename(video_path))[0]
cap = cv2.VideoCapture(video_path)

# Initialiser des listes pour stocker les données
landmarks_all_frames = []

# Définir les noms de colonnes
left_eye_columns = [f"left_eye_{i}" for i in [463, 385, 387, 263, 373, 380]]
right_eye_columns = [f"right_eye_{i}" for i in [133, 158, 160, 33, 144, 153]]
mouth_columns = [f"mouth_{i}" for i in [78, 82, 312, 308, 317, 87]]

# Créer le fichier CSV et écrire l'en-tête
csv_file_path = f"{video_name}_landmarks.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Écrire l'en-tête du fichier CSV avec toutes les colonnes
    writer.writerow(left_eye_columns + right_eye_columns + mouth_columns)

# Lecture de la vidéo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Détecter les visages dans l'image
    faces = face_mesh.process(frame)

    if faces.multi_face_landmarks:
        for face_landmarks in faces.multi_face_landmarks:
            # Extraire les coordonnées des points des yeux et de la bouche
            left_eye_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [463, 385, 387, 263, 373, 380]]
            right_eye_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [133, 158, 160, 33, 144, 153]]
            mouth_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [78, 82, 312, 308, 317, 87]]

            # Ajouter les coordonnées aux listes respectives
            landmarks_all_frames.append(left_eye_landmarks + right_eye_landmarks + mouth_landmarks)

            # Dessiner les points des yeux sur l'image
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=drawing_spec)

    # Afficher la vidéo
    cv2.imshow('Video', frame)

    # Arrêter la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# En dehors de la boucle, écrire les coordonnées dans le fichier CSV
with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    # Écrire les coordonnées dans le fichier CSV
    writer.writerows(landmarks_all_frames)
    print(f"Les coordonnées des landmarks ont été sauvegardées dans : {csv_file_path}")

# Libérer la capture vidéo
cap.release()
cv2.destroyAllWindows()


: 

## b) Récupérer les coordonnées(x,y) de plusieurs vidéo dans plusieurs csv (à modifier)
Il faudrait rajouter la coordonnée z

In [1]:
import cv2
import mediapipe as mp
import csv
import os

# Ajoutez cette ligne pour spécifier la version de numpy
import numpy as np; np_version = np.__version__.split('.')

if int(np_version[0]) < 1 or (int(np_version[0]) == 1 and int(np_version[1]) < 20):
    raise ImportError("Numpy version 1.20.0 or above is required for this version of mediapipe. "
                      "Please upgrade numpy by running: pip install --upgrade numpy")

# Charger le modèle Mediapipe pour la détection des yeux et des lèvres
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

def process_video(video_path):
    # Initialiser le détecteur de visage et le dessinateur pour les annotations
    face_mesh = mp_face_mesh.FaceMesh()
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Initialiser des listes pour stocker les données
    landmarks_all_frames = []

    # Définir les noms de colonnes
    left_eye_columns = [f"left_eye_{i}" for i in [463, 385, 387, 263, 373, 380]]
    right_eye_columns = [f"right_eye_{i}" for i in [133, 158, 160, 33, 144, 153]]
    mouth_columns = [f"mouth_{i}" for i in [78, 82, 312, 308, 317, 87]]

    # Créer le fichier CSV et écrire l'en-tête
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_file_path = f"{video_name}_landmarks.csv"
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Écrire l'en-tête du fichier CSV avec toutes les colonnes
        writer.writerow(left_eye_columns + right_eye_columns + mouth_columns)

    # Charger la vidéo
    cap = cv2.VideoCapture(video_path)

    # Lecture de la vidéo
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir l'image en niveaux de gris
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Détecter les visages dans l'image
        faces = face_mesh.process(frame)

        if faces.multi_face_landmarks:
            for face_landmarks in faces.multi_face_landmarks:
                # Extraire les coordonnées des points des yeux et de la bouche
                left_eye_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [463, 385, 387, 263, 373, 380]]
                right_eye_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [133, 158, 160, 33, 144, 153]]
                mouth_landmarks = [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in [78, 82, 312, 308, 317, 87]]

                # Ajouter les coordonnées aux listes respectives
                landmarks_all_frames.append(left_eye_landmarks + right_eye_landmarks + mouth_landmarks)

                # Dessiner les points des yeux sur l'image
                mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=drawing_spec)

        # Afficher la vidéo
        cv2.imshow('Video', frame)

        # Arrêter la boucle si la touche 'q' est pressée
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # En dehors de la boucle, écrire les coordonnées dans le fichier CSV
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Écrire les coordonnées dans le fichier CSV
        writer.writerows(landmarks_all_frames)
        print(f"Les coordonnées des landmarks ont été sauvegardées dans : {csv_file_path}")

    # Libérer la capture vidéo
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()


# Liste des vidéos à traiter
video_paths = ['kss#1-3#F#rldd#10-0.mp4', 'kss#8-9#F#rldd#28-10.mp4','kss#1-3#F#rldd#10-0_2.mp4']

# Traiter chaque vidéo
for video_path in video_paths:
    process_video(video_path)
print("Terminé")


I0000 00:00:1702083568.907194       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Les coordonnées des landmarks ont été sauvegardées dans : kss#1-3#F#rldd#10-0_landmarks.csv


I0000 00:00:1702083587.753498       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


Les coordonnées des landmarks ont été sauvegardées dans : kss#8-9#F#rldd#28-10_landmarks.csv


I0000 00:00:1702083606.436799       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


Les coordonnées des landmarks ont été sauvegardées dans : kss#1-3#F#rldd#10-0_2_landmarks.csv
Terminé


## 2 ème étape : Statistiques sur nos données  (ne pas modifier)

In [ ]:
import pandas as pd
import numpy as np

# Chemins des fichiers CSV
left_eye_csv_path = "output_video_opencv_left_eye_landmarks.csv"
right_eye_csv_path = "output_video_opencv_mouth_landmarks.csv"
mouth_csv_path = "output_video_opencv_right_eye_landmarks.csv"

# Liste des chemins des fichiers
csv_paths = [left_eye_csv_path, right_eye_csv_path, mouth_csv_path]

# Dictionnaire pour stocker les résultats
results = {}

# Parcourir chaque fichier CSV
for csv_path in csv_paths:
    # Lire le fichier CSV
    df = pd.read_csv(csv_path, header=None)

    # Récupérer le nom du fichier à partir du chemin
    file_name = csv_path.split("/")[-1]

    # Initialiser le dictionnaire pour le fichier actuel
    results[file_name] = {}

    # Récupérer les noms de colonnes à partir de la première ligne
    column_names = df.iloc[0]

    # Parcourir chaque colonne du dataframe
    for col, col_name in zip(df.columns, column_names):
        # Ignorer la première ligne (en-tête)
        values = df[col][1:].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else x)

        # Vérifier si chaque valeur est une liste avant d'accéder à ses éléments
        x_values = values.apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        y_values = values.apply(lambda x: x[1] if isinstance(x, (list, np.ndarray)) else x)

        # Stocker les résultats pour la colonne actuelle avec le nom de l'en-tête
        results[file_name][col_name] = {
            "min_x": x_values.min(),
            "max_x": x_values.max(),
            "mean_x": x_values.mean(),
            "min_y": y_values.min(),
            "max_y": y_values.max(),
            "mean_y": y_values.mean(),
        }

# Afficher les résultats
for file_name, values in results.items():
    print(f"Résultats pour le fichier {file_name}:")
    for col_name, stats in values.items():
        print(f"{col_name}:")
        print(f"  Min X: {stats['min_x']}")
        print(f"  Max X: {stats['max_x']}")
        print(f"  Moyenne X: {stats['mean_x']}")
        print(f"  Min Y: {stats['min_y']}")
        print(f"  Max Y: {stats['max_y']}")
        print(f"  Moyenne Y: {stats['mean_y']}")
        print()


: 

## 3 ème étape : Créer un data frame avec les valeurs de  EAR et MAR (à modifier)
--

A partir du csv créee pour les vidéos ['kss#1-3#F#rldd#10-0_landmarks.csv', 'kss#8-9#F#rldd#29-10_landmarks.csv', 'kss#8-9#F#rldd#28-10_landmarks.csv'], calculer pour chaque frame EARdroit, EARgauche, EARmoyen et Mar et les mettre dans un dataframe final.  Dans le dataframe final, on veut que chaque ligne représente 1 vidéo et dans les colonnes on a : 
voici ce qu'on met dans les colonnes [nomdelavidéo (c'est-à-dire par exemple 'kss#1-3#F#rldd#10-0') , EARgauche_0 (pour la frame 0 ), …,EARgauche_875 (pour la frame 875 ),EARdroit_0 (pour la frame 0 ), …,EARdroit_875 (pour la frame 875 ),EARmoyen0 (pour la frame 0 ), …,EARmoyen_875 (pour la frame 875 ),MAR_0 (pour la frame 0 ), …,MAR_875 (pour la frame 875 )] . Si j'ai 3 vidéos , j'aurais donc un dataframe avec 4 lignes (1 ligne pour les entête et chaque ligne suivante représente 1 vidéo)  et 4*875=4375 colonnes car on a 875 frames.

In [3]:
import pandas as pd
import numpy as np

# Fonctions EAR et MAR
def eye_aspect_ratio(eye):
    A = np.linalg.norm(np.array(eval(eye[1])) - np.array(eval(eye[5])))
    B = np.linalg.norm(np.array(eval(eye[2])) - np.array(eval(eye[4])))
    C = np.linalg.norm(np.array(eval(eye[0])) - np.array(eval(eye[3])))
    ear = (A + B) / (2.0 * C)
    return ear

def EAR_mean(EAR1, EAR2):
    return (EAR1 + EAR2) / 2

def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(np.array(eval(mouth[1])) - np.array(eval(mouth[5])))
    B = np.linalg.norm(np.array(eval(mouth[2])) - np.array(eval(mouth[4])))
    C = np.linalg.norm(np.array(eval(mouth[0])) - np.array(eval(mouth[3])))
    mar = (A + B) / (2.0 * C)
    return mar

# Charger les données à partir des fichiers CSV
# Liste des vidéos à traiter
video_names = ['kss#1-3#F#rldd#10-0', 'kss#8-9#F#rldd#28-10', 'kss#1-3#F#rldd#10-0_2']

video_paths = [name + '.mp4' for name in video_names]

# Créer une liste video_files avec les noms de fichiers modifiés
video_files = [name + '_landmarks.csv' for name in video_names]

signes = ["EAR_left","EAR_right", "EAR_mean","MAR"]

# Initialiser l'entête du data_frame
columns = ['Nom_video']
for video_file in video_files:
    # Charger le CSV
    df = pd.read_csv(video_file)
    taille_premiere_colonne = len(df.iloc[:, 0])

for signe in signes:
    for frame in range(taille_premiere_colonne):  # Supposant 876 frames par vidéo
        columns.append(f'{signe}_{frame}')

# Créer le DataFrame final
df_final = pd.DataFrame(columns=columns)

df_final.to_csv('resultat_final.csv', index=False)

# Parcourir chaque fichier vidéo
for video_name in video_names:
    print("Traitement en cours pour : ", video_name)

    # Charger le CSV
    df = pd.read_csv(video_name + '_landmarks.csv')

    # Récupérer le nom de la vidéo à partir du nom du fichier
    df_final.at[video_name, 'Nom_video'] = video_name

    # Calculer l'EAR pour chaque œil et ajouter les valeurs à df_final
    for frame in range(len(df)):
        eye_coordinates = df.loc[frame, ['left_eye_463', 'left_eye_385', 'left_eye_387', 'left_eye_263', 'left_eye_373', 'left_eye_380']]
        EAR = eye_aspect_ratio(eye_coordinates)

        # Ajouter la valeur EAR à la colonne correspondante dans df_final
        column_name = f'EAR_left_{frame}'
        df_final.at[video_name, column_name] = EAR


print(df_final)

# Transformer df_final en csv
df_final.to_csv('resultat_final.csv', index=False)

# Enregistrez le DataFrame final dans un fichier CSV


Traitement en cours pour :  kss#1-3#F#rldd#10-0
Traitement en cours pour :  kss#8-9#F#rldd#28-10
Traitement en cours pour :  kss#1-3#F#rldd#10-0_2
                                   Nom_video EAR_left_0 EAR_left_1 EAR_left_2   
kss#1-3#F#rldd#10-0      kss#1-3#F#rldd#10-0   0.118014   0.129428   0.134889  \
kss#8-9#F#rldd#28-10    kss#8-9#F#rldd#28-10   0.121962    0.15786   0.142293   
kss#1-3#F#rldd#10-0_2  kss#1-3#F#rldd#10-0_2   0.118014   0.129428   0.134889   

                      EAR_left_3 EAR_left_4 EAR_left_5 EAR_left_6 EAR_left_7   
kss#1-3#F#rldd#10-0     0.130209   0.129128   0.128823   0.130211   0.129761  \
kss#8-9#F#rldd#28-10    0.150228   0.155248   0.151252   0.154831   0.152732   
kss#1-3#F#rldd#10-0_2   0.130209   0.129128   0.128823   0.130211   0.129761   

                      EAR_left_8  ... MAR_866 MAR_867 MAR_868 MAR_869 MAR_870   
kss#1-3#F#rldd#10-0     0.130481  ...     NaN     NaN     NaN     NaN     NaN  \
kss#8-9#F#rldd#28-10    0.148958  ...     NaN